In [56]:
# !pip install langgraph langchain_ollama 
!pip install -qU duckduckgo-search langchain-community


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
from langchain_ollama import ChatOllama
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.prebuilt import ToolNode
search = DuckDuckGoSearchRun()


llm = ChatOllama(model="jacob-ebey/phi4-tools:latest")
# llm = ChatOllama(model="qwen2.5:32b")
# llm = ChatOllama(model="MFDoom/deepseek-r1-tool-calling:32b")

system = '''You are an AI assistant that follows a ReAct (Reasoning + Acting) framework to answer questions efficiently using search. You will think step by step, take appropriate actions, and provide observations before concluding with an answer. Follow the format strictly:

Format:
Question: The question asked.
Thought: Reason logically about how to find the answer.
Action: Choose an action. If search is needed, use Search[query]. If an answer is found, use Finish[answer].
Observation: The result of the action.
Repeat steps until the answer is found.
Example 1:
Question: What profession does Nicholas Ray and Elia Kazan have in common?
Thought 1: I need to search Nicholas Ray and Elia Kazan, find their professions, then find the profession they have in common.
Action 1: Search[Nicholas Ray]
Observation 1: Nicholas Ray (born Raymond Nicholas Kienzle Jr., August 7, 1911 - June 16, 1979) was an American film director, screenwriter, and actor best known for the 1955 film Rebel Without a Cause.
Thought 2: Professions of Nicholas Ray are director, screenwriter, and actor. I need to search Elia Kazan next and find his professions.
Action 2: Search[Elia Kazan]
Observation 2: Elia Kazan was an American film and theatre director, producer, screenwriter, and actor.
Thought 3: Professions of Elia Kazan are director, producer, screenwriter, and actor. So the profession Nicholas Ray and Elia Kazan have in common is director, screenwriter, and actor.
Action 3: Finish[director, screenwriter, actor]

Example 2:
Question: Which magazine was started first, Arthur’s Magazine or First for Women?
Thought 1: I need to search Arthur’s Magazine and First for Women, and find which was started first.
Action 1: Search[Arthur’s Magazine]
Observation 1: Arthur’s Magazine (1844-1846) was an American literary periodical published in Philadelphia in the 19th century.
Thought 2: Arthur’s Magazine was started in 1844. I need to search First for Women next.
Action 2: Search[First for Women]
Observation 2: First for Women is a women’s magazine published by Bauer Media Group in the USA. The magazine was started in 1989.
Thought 3: First for Women was started in 1989. Since 1844 (Arthur’s Magazine) < 1989 (First for Women), Arthur’s Magazine was started first.
Action 3: Finish[Arthur’s Magazine]

Follow this process for all questions. Do not assume answers without searching. Always reason through each step before deciding on an action.'''

tools = [search]
toolLLMNode = llm.bind_tools(tools)
toolNode = ToolNode(tools,)

In [27]:
toolLLMNode.invoke("what is the current news in delhi right now")

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'jacob-ebey/phi4-tools:latest', 'created_at': '2025-02-07T04:19:44.82160498Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4603630054, 'load_duration': 3833287804, 'prompt_eval_count': 182, 'prompt_eval_duration': 216000000, 'eval_count': 41, 'eval_duration': 553000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-13a230cb-7bec-453b-b070-991c394a235b-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'current news in Delhi'}, 'id': 'bbd51ad7-18c5-4291-88d3-5296360ebed0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 182, 'output_tokens': 41, 'total_tokens': 223})

In [28]:
t = toolNode.invoke({"messages": [toolLLMNode.invoke("what is the current news in delhi right now")]})
t["messages"][-1].content

'Today\'s Delhi News: Get all the Latest and Breaking Delhi News on NDTV. Live local Delhi news coverage and updates on Politics, Election, Weather, Crime and More Delhi News Live Updates: Chief Minister Atishi of the Aam Aadmi Party (AAP) Monday formally requested the Chief Election Commissioner (CEC) to conduct a fresh verification of voter deletions in the New Delhi Assembly constituency. This plea follows the Election Commission\'s release of a revised ... Delhi Farmers Protest, Schools Bomb Threat Today Latest News Updates: "On December 16, a tractor march will be organised outside Punjab, and on December 18, we have called for a \'Rail Roko\' in Punjab. We appeal to all Punjabis to participate in large numbers," Pandher said. Delhi News Live: From weather alerts to government announcements, the Delhi News Live blog provides real-time updates on everything that matters. Stay informed about the biggest stories affecting ... City Latest News Today Live: People take a dip in the Yamu

In [29]:
from typing import Sequence
import operator
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing import TypedDict,Annotated,Literal
class State(TypedDict):
    input: str
    intermediate_steps : Annotated[list,operator.add]
    final_result: str
    system_prompt: str
    iteration:int
    
from pydantic import BaseModel,Field


class Action(BaseModel):
    # reason: str = Field(description="the input of the user and how do you reason the decision")
    reason: str=Field(description="which decision was chosen and why based on the action") 
    decision: Literal["Finish", "ToolCall"] = Field(description="the action to be chosen by evaluating the thought process. if the input is undecisive then call ToolCall otherwise Finish")
actionLLM = llm.with_structured_output(Action,method="json_schema")

In [30]:
resp = actionLLM.invoke("i now have complete information now i can rest")

In [31]:
resp

Action(reason="The user's statement is vague and does not provide enough context to determine if it contains sensitive information or requires a specific action. It could be interpreted as a personal reflection rather than something that needs moderation.", decision='ToolCall')

In [32]:

# def system_init(state:State):
    # return llm.invoke(f"based on the system prompt :{state['system']}\n\n plan the steps for the following question:{state['input']}")
def thinking_llm(state:State):
    if state["iteration"] !=0:
        resp = llm.invoke(f"Analyze the situation step by step based on the following context: {state['intermediate_steps']}. Clearly reason through each step and output the what do you need to do in order to perform the lab activity safely you have the access to web search if needed use that by asking that you need to search the web for the so and so.\n \n Thought {state['iteration']} :" )
        fs = state["intermediate_steps"] + [f"Thought {state['iteration']}"+ resp.content]
        state.update({"intermediate_steps": fs})
        state.update({"iteration":state["iteration"]+1})
        
        return state
    else :
        resp = llm.invoke(f"given system_prompt [{state['system_prompt']}] and input {state['input']} output the thought. \n\n Thought {state['iteration']} :")
        fs = state["intermediate_steps"] + [f"Thought {state['iteration']}"+ resp.content]
        state.update({"intermediate_steps": fs})
        state.update({"iteration":state["iteration"]+1})
        print("thinking...")
        return state

def action_llm(state:State):
    resp = actionLLM.invoke(f"based on the following thought process should the model be calling a ToolCall or if the model has all information then it should call Finish. Based on the input {state['intermediate_steps'][-1]}")
    fs = state["intermediate_steps"] + [f"Action {state['iteration']}"+ resp.reason]
    state.update({"intermediate_steps": fs})
    # state.update({"iteration":state["iteration"]+1})
    if resp.decision == "Finish":
        print("choosing action to finish")
        return "FinishingAgent"
    elif resp.decision == "ToolCall":
        print("choosing action to use Tools")
        return "ToolNode"
    
def finishingNode(state:State):
    resp = llm.invoke(f"based on '{state['input']}' which is the user prompt and the LLMs Thought process [{state['intermediate_steps']}] generate a final response for the input prompt using the information from Thought Process.")
    # fs = state["intermediate_steps"] + [resp.content]
    state.update({"final_result": [resp.content]})
    print("finished!")
    return state

def toolingLLMNode(state:State):

    resp = toolLLMNode.invoke(f'based on {str(state["intermediate_steps"][-2:])} call the appropriate tool')
    respToTool = toolNode.invoke({"messages":[resp]})
    fs = state["intermediate_steps"] + [f"observation {state['iteration']}: "+respToTool["messages"][-1].content]
    state.update({"intermediate_steps": fs})
    print("running Tool")
    return state

In [33]:
from langgraph.graph import StateGraph,START,END
build = StateGraph(State)
build.add_node("ToolDecidingNode",toolingLLMNode)
# build.add_node("ToolNode",toolNode)
build.add_node("ThoughtNode",thinking_llm)
# build.add_node("actionNode",action_llm)
build.add_node("final_node",finishingNode)

build.add_edge(START,"ThoughtNode")
# build.add_edge("ThoughtNode","actionNode")
# build.add_edge("ToolDecidingNode","ToolNode")
build.add_edge("ToolDecidingNode", "ThoughtNode")
build.add_conditional_edges("ThoughtNode",action_llm,{"ToolCall":"ToolDecidingNode","FinishingAgent":"final_node"})
build.add_edge("final_node",END)
# build.set_entry_point(START)
# build.set_finish_point(END)



In [34]:
experiment_name = [
    "To verify the relationship between voltage (V), current (I), and resistance (R) in an electrical circuit, as expressed by Ohm's Law: V=IRV = IRV=IR",
    "To apply Kirchhoff’s Voltage Law (KVL) and Kirchhoff’s Current Law (KCL) to analyze and validate simple electrical circuits.",
    "To study the behavior of resistors in series and parallel configurations, including the equivalent resistance calculation.",
    "To study the forward and reverse bias characteristics of a PN junction diode.",
    "To analyze voltage regulation using a Zener diode.",
    "To construct and analyze the performance of half-wave and full-wave rectifiers, both with and without filters.",
    "To analyze and implement wave-shaping circuits using diodes for clipping and clamping applications.",
    "To analyze the input and output characteristics of Bipolar Junction Transistors (BJTs) and Field-Effect Transistors (FETs).",
    "To design a common emitter amplifier and analyze its frequency response.",
    "To implement and analyze operational amplifier (Op-Amp) circuits: inverting, non-inverting, summing, and differentiator configurations.",
    "To verify the operation of basic logic gates: AND, OR, NOT, NAND, NOR, XOR, and XNOR",
    "To investigate the charging and discharging behavior of a capacitor in an RC circuit and understand the time constant.",
    "To investigate the resonance behavior of an LC circuit and measure the resonant frequency.",
    "To calibrate an oscilloscope for accurate measurements of voltage and time.",
    "To determine the turns ratio of a transformer and verify the relationship between the primary and secondary voltages.",
    "To construct and analyze a bridge rectifier circuit and compare its performance with a half-wave rectifier.",
    "To study the frequency response of a low-pass filter and determine its cutoff frequency.",
    "To measure the input and output impedances of an operational amplifier (Op-Amp) in a given configuration.",
    "To generate and analyze Lissajous figures using an oscilloscope by applying two sinusoidal signals with different frequencies.",
    "To study the magnetic field produced by a solenoid and verify the relationship between current and magnetic field strength."
]

In [35]:
import copy
remaining_exp = copy.deepcopy(experiment_name)

In [36]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

counting = 1
for i in range(len(remaining_exp)):
    graph = build.compile(checkpointer=memory)
    config = {"configurable": {"thread_id": str(counting)}}
    exp_name = remaining_exp.pop()
    for event  in graph.stream({'input':f"plan the following experiment {{{exp_name}}} and ensure that the experiments are with in the safety parameters also ensure that the output contains items required and experiments steps",'intermediate_steps':[],'iteration':0,'system_prompt':system},config=config):
        if "final_node" in event:
            print(counting)
            with open("experiments_generated_phi4.md",'a',encoding="utf-8") as F:
                F.write("\n---\n")
                F.write(f"# {counting} {exp_name}\n")
                F.write(event['final_node']['final_result'][0])
            with open("experiments_thoughts_phi4.md",'a',encoding="utf-8") as F:
                F.write("\n---\n")
                F.write(f"# {counting} {exp_name}\n")
                F.write(event['final_node']['intermediate_steps'][0])
            counting+=1

thinking...
choosing action to finish
finished!
1
thinking...
choosing action to finish
finished!
2
thinking...
choosing action to finish
finished!
3
thinking...
choosing action to finish
finished!
4
thinking...
choosing action to finish
finished!
5
thinking...
choosing action to finish
finished!
6
thinking...
choosing action to finish
finished!
7
thinking...
choosing action to finish
finished!
8
thinking...
choosing action to finish
finished!
9
thinking...
choosing action to finish
finished!
10
thinking...
choosing action to finish
finished!
11
thinking...
choosing action to finish
finished!
12
thinking...
choosing action to finish
finished!
13
thinking...
choosing action to finish
finished!
14
thinking...
choosing action to finish
finished!
15
thinking...
choosing action to finish
finished!
16
thinking...
choosing action to finish
finished!
17
thinking...
choosing action to finish
finished!
18
thinking...
choosing action to finish
finished!
19
thinking...
choosing action to finish
fi

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass